In [1]:
import numpy as np
from gurobipy import Model, GRB, quicksum

# Problem 1

This is not my solution, this solver was to check it. My proper solution is submitted as another file.

In [42]:
c_ij = np.array([[15, 10, 8, 12], # Rows DCs (supply), columns stores (demand)
        [9, 11, 7, 5],
        [13, 14, 3, 4]])
supply = [200, 150, 225]
demand = [140, 130, 165, 100]

In [41]:
model = Model("Shipping Problem")
X = model.addVars(len(supply), len(demand), name="X", lb=0)
model.setObjective(
    quicksum(c_ij[i][j] * X[i, j] for i in range(len(supply)) for j in range(len(demand))),
    GRB.MINIMIZE
)
for i in range(len(supply)):
    model.addConstr(quicksum(X[i, j] for j in range(len(demand))) <= supply[i], name=f"supply_{i}")
for j in range(len(demand)):
    model.addConstr(quicksum(X[i, j] for i in range(len(supply))) == demand[j], name=f"demand_{j}")

model.optimize()
for v in model.getVars():
    print(f"{v.varName}: {v.x}")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 5625U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x7fdde0d1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve time: 0.02s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4550000e+03   4.000000e+01   0.000000e+00      0s
       2    3.6150000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  3.615000000e+03
X[0,0]: 0.0
X[0,1]: 130.0
X[0,2]: 30.0
X[0,3]: 0.0
X[1,0]: 140.0
X[1,1]: 0.0
X[1,2]: 0.

# Problem 2

In [29]:
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
p2_distances = np.array([[0, 61, 30, np.inf, np.inf, np.inf, np.inf, np.inf],
                      [61, 0, 68, 46, np.inf, np.inf, np.inf, np.inf],
                      [30, 68, 0, 97, 43, np.inf, 60, np.inf],
                      [np.inf, 46, 97, 0, 73, 30, np.inf, np.inf],
                      [np.inf, np.inf, 43, 73, 0, 66, 22, np.inf],
                      [np.inf, np.inf, np.inf, 30, 66, 0, 70, 26],
                      [np.inf, np.inf, 60, np.inf, 22, 70, 0, 80],
                      [np.inf, np.inf, np.inf, np.inf, np.inf, 26, 80, 0]])



def djikstra(nodes, distances, root):
    predecessors = [0] * distances.shape[1]
    root_distances = [distances[nodes.index(root)][:]]
    for i in range(distances.shape[1]):
        if distances [nodes.index(root)][i] != np.inf:
            predecessors[i] = root
        for j in range(distances.shape[1]):
            if distances[nodes.index(root)][j] + distances[j][i] < distances[nodes.index(root)][i]:
                predecessors[i] = nodes[j]
                root_distances[0][i] = distances[nodes.index(root)][j] + distances[j][i]

    return root_distances, predecessors

for node in ['B', 'C', 'D', 'E']:
    print(f"Optimal distances and predecessors from node {node}:")
    opt_dist, pred = djikstra(nodes, p2_distances, node)
    print(opt_dist, pred, "\n")


Optimal distances and predecessors from node B:
[array([ 61.,   0.,  68.,  46., 111.,  76., 128., 102.])] ['B', 'B', 'B', 'B', 'C', 'D', 'C', 'F'] 

Optimal distances and predecessors from node C:
[array([ 30.,  68.,   0.,  97.,  43., 109.,  60., 135.])] ['C', 'C', 'C', 'C', 'C', 'E', 'C', 'F'] 

Optimal distances and predecessors from node D:
[array([107.,  46.,  97.,   0.,  73.,  30.,  95.,  56.])] ['B', 'D', 'D', 'D', 'D', 'D', 'E', 'F'] 

Optimal distances and predecessors from node E:
[array([ 73., 111.,  43.,  73.,   0.,  66.,  22.,  92.])] ['C', 'C', 'E', 'E', 'E', 'E', 'E', 'F'] 



# Problem 3

In [32]:
p3_distances = np.array([[0, 20, np.inf, 15, np.inf, 7, np.inf, np.inf],
                         [20, 0, 15, 6, 9, np.inf, np.inf, np.inf],
                         [np.inf, 15, 0, np.inf, 6, np.inf, np.inf, 12],
                         [15, 6, np.inf, 0, 14, 5, 13, np.inf],
                         [np.inf, 9, 6, 14, 0, np.inf, 14, 8],
                         [7, np.inf, np.inf, 5, np.inf, 0, 20, np.inf],
                         [np.inf, np.inf, np.inf, 13, 14, 20, 0, 20],
                         [np.inf, np.inf, 12, np.inf, 8, np.inf, 20, 0]])

djikstra(nodes, p3_distances, 'D')

([array([12.,  6., 20.,  0., 14.,  5., 13., 22.])],
 ['F', 'D', 'E', 'D', 'D', 'D', 'D', 'E'])

If we compare these results to the V8 and P8 matrices in the lecture notes, we can see that the values match exactly. The fourth column of V8 is exactly the distance vector above. This makes sense because it represents going from anby node (row) to the fourth node D (column). And the fourth row of P8 is exactly the predecessor vector above. This makes sense because if we start at D and move away, then the predecessor will be the last node visited before arriving.